In [41]:
import pandas as pd
import numpy as np
import statsmodels.api as sm

df_gap = pd.read_csv("Data\Google Trends Data\GAP_google.csv",skiprows=1)
df_navy = pd.read_csv("Data\Google Trends Data\OldNavy_google.csv",skiprows=1)
df_ae = pd.read_csv("Data\Google Trends Data\AmericanEagle_google.csv",skiprows=1)
df_br = pd.read_csv("Data\Google Trends Data\BananaRepublic_google.csv",skiprows=1)

df_list = [df_gap, df_navy, df_ae, df_br]
df_names = ['df_gap', 'df_navy', 'df_ae', 'df_br']
df_br.head()

,Month,Banana Republic: (Worldwide)
0,2012-01,64
1,2012-02,67
2,2012-03,70
3,2012-04,69
4,2012-05,64


In [42]:
threshold_year = 2015

for df in df_list:
    df.columns = ['Date', 'Search Volume']
    df['Date'] = pd.to_datetime(df['Date'])
    df['Post'] = df['Date'].apply(lambda x: 1 if x.year >= 2015 else 0)
df_gap

,Date,Search Volume,Post
0,2012-01-01,70,0
1,2012-02-01,71,0
2,2012-03-01,75,0
3,2012-04-01,79,0
4,2012-05-01,74,0
...,...,...,...
116,2021-09-01,44,1
117,2021-10-01,46,1
118,2021-11-01,57,1
119,2021-12-01,48,1


In [43]:
df_gap.dtypes

Date             datetime64[ns]
Search Volume             int64
Post                      int64
dtype: object

In [44]:
# Create an empty list to store the regression models
models = []

for df in df_list:
    
    # Define the independent variables
    X = df[['Date', 'Post']]
    
    # Convert 'Date' to Unix timestamp (numeric format)
    X['Date'] = X['Date'].apply(lambda x: x.timestamp())
    
    # Create an interaction term between 'Date' and 'Post'
    X['Date_Post_Interact'] = X['Date'] * X['Post']
    
    # Add a constant term to the independent variables
    X = sm.add_constant(X)
    
    # Define the dependent variable
    y = df['Search Volume']
    
    # Fit the linear regression model
    model = sm.OLS(y, X).fit()
    
    # Append the model to the list of models
    models.append(model)


<ipython-input-44-fb888ff2b8cb>:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X['Date'] = X['Date'].apply(lambda x: x.timestamp())


In [45]:
for i, model in enumerate(models):
    print(f"Regression Results for DataFrame: " + df_names[i])
    print(model.summary())
    print("\n")

Regression Results for DataFrame: df_gap
                            OLS Regression Results                            
Dep. Variable:          Search Volume   R-squared:                       0.708
Model:                            OLS   Adj. R-squared:                  0.701
Method:                 Least Squares   F-statistic:                     94.76
Date:                Sun, 01 Oct 2023   Prob (F-statistic):           3.57e-31
Time:                        16:24:17   Log-Likelihood:                -431.97
No. Observations:                 121   AIC:                             871.9
Df Residuals:                     117   BIC:                             883.1
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
                         coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------